In [1]:
from pyoxigraph import *
import chromadb
from RDF import RDF
from OWL import OWL
from RDFS import RDFS
from Nodes import *
from Graph import Ontology

In [2]:
file = "/home/kipp_sa/github/EmbedAlign/test/bfo-core.owl"
bfo = Ontology.load(file = "../test/bfo-core.owl")
Client = chromadb.Client()
label_collection = Client.get_or_create_collection(name="labels")
structure_collection = Client.get_or_create_collection(name = "Structure")

In [3]:
for c in bfo.classes:
    print(c)


temporal instant
temporal interval
history
zero-dimensional temporal region
fiat point
fiat surface
relational quality
(fiat line OR fiat point)
fiat line
immaterial entity
continuant fiat boundary
(site OR material entity OR continuant fiat boundary)
material entity
(one-dimensional temporal region OR zero-dimensional temporal region)
one-dimensional temporal region
(process OR process boundary)
(process OR process boundary)
process boundary
function
generically dependent continuant
object
(site OR continuant fiat boundary)
(site OR material entity)
site
three-dimensional spatial region
object aggregate
(zero-dimensional spatial region OR one-dimensional spatial region)
one-dimensional spatial region
fiat object part
role
specifically dependent continuant
quality
zero-dimensional spatial region
realizable entity
disposition
(process OR process boundary)
process
spatiotemporal region
(two-dimensional spatial region OR zero-dimensional spatial region OR one-dimensional spatial region)
t

In [4]:
file = "/home/kipp_sa/github/EmbedAlign/test/TimeOntology.rdf"
cco_time = Ontology.load( file = file)
for c in cco_time.classes:
    print(c)

Multi-Year Temporal Interval
Night
Multi-Minute Temporal Interval
Multi-Second Temporal Interval
Reference Time
Evening
Decade
Hour
Calendar Week
Second
Calendar Day
Year
Multi-Week Temporal Interval
Day
Afternoon
Calendar Year
Week
Morning
Julian Day
Modified Julian Date
Gregorian Day
Gregorian Year
Julian Date
Multi-Month Temporal Interval
Calendar Month
Julian Year
Month
Time of Day
Multi-Day Temporal Interval
Axial Rotation Period
Unix Temporal Instant
Minute
Multi-Hour Temporal Interval


In [6]:
file = "/home/kipp_sa/github/EmbedAlign/test/time.rdf"
time_w3c = Ontology.load(file = file)
for c in time_w3c.classes:
    print(c)

Year
Time Zone
((<http://www.w3.org/2006/time#nominalPosition> CARDINALITY "1"^^<http://www.w3.org/2001/XMLSchema#integer>) OR (<http://www.w3.org/2006/time#numericPosition> CARDINALITY "1"^^<http://www.w3.org/2001/XMLSchema#integer>))
Time position
Temporal unit
Temporal position
(Time instant OR Time interval)
Temporal entity
Temporal duration
Temporal Reference System
Proper interval
Month of year
January
Time interval
Time instant
Generalized duration description
Generalized date-time description
Duration description
Time duration
Day of week
Date-time interval
Date-Time description
(Time duration OR Generalized date-time description)
(Generalized duration description OR Temporal position)


In [10]:
store = time_w3c.store
results = store.quads_for_pattern(None, RDF.type, OWL.Class, None)
subs = [r.subject for r in results]
for r in subs:
    try:
        c = map_class_type(r,store)
        print(f"class : {c}")
    except:
        print(f"no information found for {r}")

class : Year
class : Time Zone
[<NamedNode value=http://www.w3.org/2002/07/owl#unionOf>]
[<NamedNode value=http://www.w3.org/2002/07/owl#unionOf>]
[<NamedNode value=http://www.w3.org/2002/07/owl#onProperty>]
no information found for _:d39bf05983371bf21bb162732ca6f10e
class : Time position
class : Temporal unit
class : Temporal position
[<NamedNode value=http://www.w3.org/2002/07/owl#unionOf>]
[<NamedNode value=http://www.w3.org/2002/07/owl#unionOf>]
class : (Time instant OR Time interval)
class : Temporal entity
class : Temporal duration
class : Temporal Reference System
class : Proper interval
class : Month of year
class : January
class : Time interval
class : Time instant
class : Generalized duration description
class : Generalized date-time description
class : Duration description
class : Time duration
class : Day of week
class : Date-time interval
class : Date-Time description
[<NamedNode value=http://www.w3.org/2002/07/owl#unionOf>]
[<NamedNode value=http://www.w3.org/2002/07/owl#

In [ ]:
def class_loss(oc : NamedClass | ComplexClass, onto : Ontology):
    tested = []
    for c in onto.classes:
        if c == oc:
            pass
        elif c.isSubClassOf(oc):
            pass

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

/home/kipp_sa/github/EmbedAlign/embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vec_car = model.encode("car")
vec_auto = model.encode("auto")
print(model.similarity(vec_car, vec_auto))

tensor([[0.5096]])


In [3]:
vec_car = model.encode("car is a vehicle")
vec_auto = model.encode("auto is a completion method")
print(model.similarity(vec_car, vec_auto))

tensor([[0.1920]])
